In [6]:
import transformers
print(transformers.__version__)

import torch
print(torch.__version__)



4.35.2
2.1.1


In [20]:
import nltk
print(nltk.__version__)
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import torch


nltk.download('stopwords')
# Load English stopwords
stop_words = set(stopwords.words('english'))

# Read file content
with open("./Copy-Cat-Project-main/Steve Jobs at Apple’s WWDC 1997.txt", 'r') as file:
    text = file.read()

# Tokenize words
word_tokens = word_tokenize(text)

# Remove stopwords
filtered_text = [w for w in word_tokens if not w in stop_words]

# Reassemble the processed text into a string
filtered_text = ' '.join(filtered_text)

# Write the processed text into a new file
with open("./filtered_Steve_Jobs_Speech.txt", 'w') as file:
    file.write(filtered_text)

#---------------------------------------------

# Load model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Prepare dataset and data collator
dataset = TextDataset(tokenizer=tokenizer, file_path="./filtered_Steve_Jobs_Speech.txt", block_size=128)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Set training parameters
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    num_train_epochs=4,
    per_device_train_batch_size=4,
    save_steps=1000,
    save_total_limit=2,
)

# Train the model
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

trainer.train()




3.8.1


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\njjia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
100%|██████████| 12/12 [00:01<00:00,  8.75it/s]

{'train_runtime': 1.3678, 'train_samples_per_second': 35.092, 'train_steps_per_second': 8.773, 'train_loss': 5.429766337076823, 'epoch': 4.0}


TrainOutput(global_step=12, training_loss=5.429766337076823, metrics={'train_runtime': 1.3678, 'train_samples_per_second': 35.092, 'train_steps_per_second': 8.773, 'train_loss': 5.429766337076823, 'epoch': 4.0})

In [21]:
# Prepare the initial prompt for generating text
# prompt_text = "Your new prompt text here."  # Change the text here
prompt_text = "I want to live five hundred more years."

# Encode input text and add end-of-sequence token
input_ids = tokenizer.encode(prompt_text, return_tensors='pt')

# Move the model to the correct computing device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
input_ids = input_ids.to(device)

# Generate text
model.eval()
with torch.no_grad():
    output = model.generate(
        input_ids,
        max_length=200,  # Adjust the length of the generated text as needed
        num_return_sequences=10,  # Generate 10 sentences
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,  # Enable sampling strategy
        top_k=0,  # Use global sampling
        temperature=0.7  # Lower the sampling temperature to make the output more deterministic
    )

# Decode and print the generated text
for i in range(10):  # Print 10 sentences
    generated_text = tokenizer.decode(output[i], skip_special_tokens=True)
    print(generated_text)

# Save the generated text to a file
with open('./results.txt', 'w', encoding='utf-8') as file:
    for i in range(10):
        generated_text = tokenizer.decode(output[i], skip_special_tokens=True)
        file.write(generated_text + '\n\n')


I want to live five hundred more years. I want to know how I got it," he said. "I want to know how I made this life work. I want to know how I got my brain to work with the rules. I want to know how I got them to work. I want to know how I managed to fail in life. I want to know how I got lucky."

What happened next was little more than the beginning of a roller coaster ride. A couple years later, he was diagnosed with Parkinson's disease and died in 1965.

"What really got him through it was when I was diagnosed with Parkinson's disease," he said. "He was probably one of the longest-lived people my family had ever met. I had a dream. I would love to work with anyone with Parkinson's. I'd love to work with anybody with Alzheimer's disease, but I would love to work with anyone with Alzheimer's disease. I would love to work with
I want to live five hundred more years. I want to die, I want to be buried, I want to be married, I want to die my last day. I want to live another ten thousand 

In [91]:
import nltk
print(nltk.__version__)

3.8.1


In [24]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize
nltk.download('punkt')

# This is your generated text
# generated_text = """

# """

# Read the generated text from results.txt
with open('./results.txt', 'r', encoding='utf-8') as file:
    generated_text = file.read()

# # This is your reference text
# reference_texts = [
#     "Your reference text here."  # Change the text here
# ]

# Read the reference text
with open('./Copy-Cat-Project-main/Steve Jobs at Apple’s WWDC 1997.txt', 'r') as f:
    reference_texts = [line.strip() for line in f]

# Tokenize the generated and reference texts
tokenized_generated_text = word_tokenize(generated_text)
tokenized_reference_texts = [word_tokenize(ref) for ref in reference_texts]

# Calculate the BLEU score
bleu_score = sentence_bleu(tokenized_reference_texts, tokenized_generated_text)

print("BLEU score:", bleu_score)


# Calculate Perplexity
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import torch
import math

# Load the model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Prepare the dataset and data collator
dataset = TextDataset(tokenizer=tokenizer, file_path="./results.txt", block_size=128)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Create evaluation dataset
eval_dataset = TextDataset(tokenizer=tokenizer, file_path="./results.txt", block_size=128)

# Set training parameters
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    num_train_epochs=4,
    per_device_train_batch_size=4,
    save_steps=1000,
    save_total_limit=2,
)

# Train the model
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    eval_dataset=eval_dataset,  # Add this line
)

trainer.train()

# Calculate Perplexity
print("Perplexity:", math.exp(trainer.evaluate()["eval_loss"]))


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\njjia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


BLEU score: 0.06105978863487787


100%|██████████| 16/16 [00:01<00:00,  8.93it/s]


{'train_runtime': 1.792, 'train_samples_per_second': 33.482, 'train_steps_per_second': 8.929, 'train_loss': 1.6935263872146606, 'epoch': 4.0}


100%|██████████| 2/2 [00:00<00:00, 30.22it/s]

Perplexity: 3.6799107994655182
